In [1]:
import os
import cv2
import tensorflow as tf
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize, to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

In [2]:
image_directory = 'sorted_images/'

In [3]:
dataset = []
label = []
point_like_corneal_ulcers = os.listdir(image_directory+'0/')
point_flaky_mixed_corneal_ulcers = os.listdir(image_directory+'1')
flaky_corneal_ulcers = os.listdir(image_directory+'2/')


In [4]:
INPUT_SIZE = 128
for i, image_name in enumerate(point_like_corneal_ulcers):
    if(image_name.split('.')[1]=='jpg'):
        image = cv2.imread(image_directory+'0/'+image_name)
        image = Image.fromarray(image,'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)
        
for i, image_name in enumerate(point_flaky_mixed_corneal_ulcers):
    if(image_name.split('.')[1]=='jpg'):
        image = cv2.imread(image_directory+'1/'+image_name)
        image = Image.fromarray(image,'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)
        
for i, image_name in enumerate(flaky_corneal_ulcers):
    if(image_name.split('.')[1]=='jpg'):
        image = cv2.imread(image_directory+'2/'+image_name)
        image = Image.fromarray(image,'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(2)

In [5]:
dataset = np.array(dataset)
label = np.array(label)

In [6]:
x_train,x_test,y_train,y_test = train_test_split(dataset, label, test_size = 0.2)

In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(569, 128, 128, 3)
(143, 128, 128, 3)
(569,)
(143,)


In [8]:
y_train = to_categorical(y_train, num_classes = 3)
y_test = to_categorical(y_test, num_classes = 3)

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3),  kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x_train,y_train, batch_size = 64, verbose = 1, epochs = 100, validation_data = (x_test, y_test), shuffle = False)
model.save('Ulcer.h5')




Epoch 1/100


9/9 [==============================] - 4s 304ms/step - loss: 42.2769 - accuracy: 0.4183 - val_loss: 1.7826 - val_accuracy: 0.5455
Epoch 2/100
9/9 [==============================] - 2s 260ms/step - loss: 2.2315 - accuracy: 0.4728 - val_loss: 1.0159 - val_accuracy: 0.5385
Epoch 3/100
9/9 [==============================] - 2s 270ms/step - loss: 1.0030 - accuracy: 0.4763 - val_loss: 0.8773 - val_accuracy: 0.4825
Epoch 4/100
9/9 [==============================] - 3s 281ms/step - loss: 1.0688 - accuracy: 0.4886 - val_loss: 0.9989 - val_accuracy: 0.5734
Epoch 5/100
9/9 [==============================] - 2s 264ms/step - loss: 0.9659 - accuracy: 0.5114 - val_loss: 0.8517 - val_accuracy: 0.5664
Epoch 6/100
9/9 [==============================] - 2s 271ms/step - loss: 0.9107 - accuracy: 0.5290 - val_loss: 0.8999 - val_accuracy: 0.6224
Epoch 7/100
9/9 [==============================] - 2s 265ms/step - loss: 0.8206 - accuracy: 0.5905 - val_loss: 0.8785 - val_accuracy: 0.6643
Epoch 8

E:\Softwares\Anaconda\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
Model_loss, Model_accuracy = model.evaluate(x_test, y_test)

print(f"Model accuracy: {Model_accuracy}")
print(f'Model loss: {Model_loss}')

5/5 [==============================] - 0s 31ms/step - loss: 2.6662 - accuracy: 0.6294
Model accuracy: 0.6293706297874451
Model loss: 2.666245222091675


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 activation (Activation)     (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 61, 61, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                   

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [13]:
flat_dataset = [image.flatten() for image in dataset]

In [14]:
x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(flat_dataset, label, test_size=0.2)


In [15]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [17]:
rf_model.fit(x_train_rf, y_train_rf)


RandomForestClassifier(random_state=42)

In [21]:
y_pred_rf = rf_model.predict(x_test_rf)
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

Random Forest Accuracy: 0.6993006993006993
